In [1]:
#種類のフォルダ名は適宜変えてください
# folder = "..\\..\\学習データ\\追加データ\\整理\\annotation\\7_さくらんぼ"
#folder = "\\172.18.21.187\\aiコンテスト2020\\100_開発\\学習データ\\追加データ\\お試し\\annotation\\1_リンゴ"
# folder = "..\\..\\学習データ\\水増しデータ\\混合データ\\上下反転左右反転\\種類\\annotation\\1_リンゴ"
#folder = "..\\..\\学習データ\\水増しデータ\\混合データ\\v2.0Test\\彩度1.5コントラスト1.5明度1.5シャープネス1.5大きさ1.2上下反転左右反転90度回転270度回転\\品種\\annotation\\1_リンゴ"
folder = "..\\..\\学習データ\\ネット\\品種\\annotation\\6_桃"
#folder = "..\\..\\学習データ\\水増しデータ\\混合データ\\上下反転左右反転\\品種\\annotation\\1_リンゴ"

In [2]:
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import pathlib

In [3]:
#指定したファイルから左上右下、種類番号のリストのリストを返す関数
def get_box_list(file):
    f = open(file)
    #左上右下、種類番号のリストのリストを保存
    box_list = []
    for line in f:
        box = line.split(",")
        #改行削除
        box[4] = box[4][:-1]
        #print(box)
        box_list.append(box)
    return box_list

In [4]:
# 引数　座標のリスト[[左, 上, 右, 下, 種類の番号], [左, 上, 右, 下, 種類の番号], [左, 上, 右, 下, 種類の番号], ... ]
# listの名前をcoordinates_listとする
# txtのパス？？？
def make_xml(coordinates_list, txt_path):

    annotation = ET.Element('annotation')
    folder = ET.SubElement(annotation, 'folder')

    # TODO:ここを種類の名前にする パスの2つ上を見る
    #folder.text = "1_あいかの香り"
    folder.text = txt_path.split("\\")[-2]

    filename = ET.SubElement(annotation, 'filename')

    # TODO: 画像のファイル名にする pathからとってくる
    #画像の名前に変換
    image_path = txt_path.replace("annotation","image").replace(".txt", ".jpg")
    filename.text = image_path.split("\\")[-1]

    path = ET.SubElement(annotation, 'path')

    # TODO: pathの変更 \を入力するには\\とする？
    #ここは絶対パス？
    #path.text = txt_path
    #print(image_path)
    path.text = "\\\\172.18.21.187\\aiコンテスト2020\\100_開発\\学習データ\\" + image_path.split("\\学習データ\\")[1]

    source = ET.SubElement(annotation, 'source')
    database = ET.SubElement(source, 'database')
    database.text = "Unknown"
    size = ET.SubElement(annotation, 'size')
    width = ET.SubElement(size, 'width')
    height = ET.SubElement(size, 'height')
    depth = ET.SubElement(size, 'depth')

    
    #画像の大きさを求める
    img = Image.open(image_path)
    #width, height = img.size
    
    # TODO: width, height, depthの変更, 画像幅、高さ、深さ？
    width.text = str(img.size[0])
    height.text = str(img.size[1])
    depth.text = '3'

    segmented = ET.SubElement(annotation, 'segmented')

    # TODO: segmentedの変更

    segmented.text = '0'

    for oneimg_list in coordinates_list:
        object = ET.SubElement(annotation, 'object')
        name = ET.SubElement(object, 'name')
        pose = ET.SubElement(object, 'pose')
        truncated = ET.SubElement(object, 'truncated')
        difficult = ET.SubElement(object, 'difficult')

        # nameの変更
        name.text = txt_path.split("\\")[-3].split("_")[1]
        pose.text = "Unspecified"
        truncated.text = '0'
        difficult.text = '0'

        bndbox = ET.SubElement(object, 'bndbox')
        xmin = ET.SubElement(bndbox, 'xmin')
        ymin = ET.SubElement(bndbox, 'ymin')
        xmax = ET.SubElement(bndbox, 'xmax')
        ymax = ET.SubElement(bndbox, 'ymax')

        # xmin, ymin, xmax, ymaxの変更

        xmin.text = str(oneimg_list[0])
        ymin.text = str(oneimg_list[1])
        xmax.text = str(oneimg_list[2])
        ymax.text = str(oneimg_list[3])

    #XML文字列の出力
    #ET.dump(annotation)

    #XMLファイルの生成
    tree = ET.ElementTree(annotation)
    #保存先どこ
    save_folder = txt_path.replace(txt_path.split("\\")[-1], "")
    fl = save_folder + filename.text.replace(".jpg", ".xml")  
    tree.write(fl)

    return fl


In [5]:
#make_xml([['56', '39', '324', '323', '0']], "..\\..\\学習データ\\追加データ\\お試し\\annotation\\1_リンゴ\\1_あいかの香り\\1.aikanokaori.txt")

In [6]:
#繰り返し処理ここから

In [7]:
for file in glob.glob(folder + "\\*\\*.txt"):
    if "classes.txt" in file:
        continue
    box_list = get_box_list(file)
    #print(file)
    #print(len(box_list))
    #予測結果がないものは処理しない
    if len(box_list) != 0:
        #処理
        #print(box_list)
        #print()
        make_xml(box_list, file)